In [1]:
import pdfplumber
import pandas as pd
import time
from parsers.core.utils.converters import convert_pdf_to_data_frame_with_row_colors
from parsers.itmo.itmo_helpers import get_courses
from parsers.core.classifiers.course_types import CourseType, course_classifier_dict

In [13]:
#pdf_file_name = 'B:/Projects/team_project/media/curricula_examples/itmo/Автоматизация и управление - Automation & Control (2024, master).pdf'
pdf_file_name = "B:/Projects/team_project/media/curricula_examples/itmo/Биоинженерия (2024, bachelor).pdf"
table_settings = {'join_tolerance': 5, 'snap_tolerance': 5}
tolerance = 1
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
df = convert_pdf_to_data_frame_with_row_colors(pdf_file_name, 1, table_settings)

In [17]:
res = []

def __get_item_from_row(row, hours_ind):
    print(row)
    print(row[5])
    return {
            'parent': None,
            'name': row[5].replace('\n', ' '),
            'credits': row[6],
            'classroomHours': sum([int(item) for item in row[hours_ind:-3] if item and item.isdigit()]),
            'year': row[2],
            'type': row[0],
            'readBy': row[len(row)-3],
            'color': row[len(row)-1],
            'children': []}


def __course_tree_to_list(el: dict, override: str = '') -> list:
    res = []
    for child in el['children']:
        course_type = child['type'].lower()
        
        if override != '':
            if override == 'все' and course_type == '':
                child['type'] = override
            elif override != 'все':
                child['type'] = override
        
        res.append(__get_course_from_item(child))
        res.extend(__course_tree_to_list(child, child['type']))
    return res

def __get_course_from_item(item: dict) -> dict:
    return {key:item[key] for key in ['name', 'credits', 'classroomHours', 'year', 'type', 'readBy']}


for i in range(len(df)):
    try:
        hours_ind = list(df.iloc[i]).index('Лек')
        header_bottom = i
    except ValueError:
        continue
    break

cdf = df[header_bottom+1:]
current = None
for i, row in cdf.iterrows():
    if current:
        item = __get_item_from_row(row, hours_ind)
        
        if item['color'] is not None:
            parent = current
            
            while True:
                if item['color'] == parent['color']:
                    parent_parent = parent['parent']
                    if parent_parent is not None:
                        parent_parent['children'].append(item)
                        item['parent'] = parent_parent
                    else:
                        res.append(item)
                    break
                
                parent = parent['parent']
                
                if parent is None:
                    item['parent'] = current
                    current['children'].append(item)
                    break
        
        else:
            item['parent'] = current
            current['children'].append(item)
        
        current = item
        
    else:
        item = __get_item_from_row(row, hours_ind)
        res.append(item)
        current = item
res = {'children': res}

course_list = __course_tree_to_list(res)
course_list = [course for course in course_list if course['year']]

courses_to_add = []
for course in course_list:
    if course['type'] == 'все':
        course['type'] = course_classifier_dict[CourseType.COMPULSORY_TYPE]
    elif course['type'] == 'все/0':
        course['type'] = course_classifier_dict[CourseType.FACULTATIVE_TYPE]
    else:
        course['type'] = course_classifier_dict[CourseType.ELECTIVE_TYPE]
        
    course['classroomHours'] = int(course['classroomHours'])
    course['credits'] = int(course['credits'])
    
    if course['year'].isdigit():
        course['year'] = (int(course['year']) + 1) // 2
    else:
        sems = course['year'].replace('\n', '').split(',')
        sems = list(set([(int(sem.strip()) + 1) // 2 for sem in sems]))
        course['year'] = sems[0]
        
        for i in range(1, len(sems)):
            new_course = course.copy()
            new_course['year'] = sems[i]
            courses_to_add.append(new_course)

course_list.extend(courses_to_add)

course_dict = {}
for item in course_list:
    course_dict.setdefault((item['name']), []).append(item)

unique_courses = []

for key, items in course_dict.items():
    if len(items) > 1:
        unique = [items[0]]
        for item in items[1:]:
            is_unique = True
            for u in unique:
                if item['year'] == u['year']:
                    is_unique = False
                    
                    if item['type'] == course_classifier_dict[CourseType.COMPULSORY_TYPE]:
                        u['classroomHours'] += item['classroomHours']
                    break
            if is_unique:
                unique.append(item)
        unique_courses.extend(unique)
    else:
        unique_courses.extend(items)

0                                
1                                
2                                
3                                
4                                
5     Блок 1. Модули (дисциплины)
6                                
7                                
8                                
9                                
10                               
11                               
12                               
13                               
14                               
15                               
16                               
17                               
18                               
19                               
20                               
21                               
22                               
23                               
24                               
25                               
26                               
27                               
28                               
29            

AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
l = []

for key, items in res.items():
    if len(items) > 1:
        unique = [items[0]]
        for item in items[1:]:
            is_unique = True
            for u in unique:
                if item['year'] == u['year']:
                    is_unique = False
                    
                    if item['type'] == course_classifier_dict[CourseType.COMPULSORY_TYPE]:
                        u['classroomHours'] += item['classroomHours']
                    break
            if is_unique:
                unique.append(item)
        l.extend(unique)
    else:
        l.extend(items)

l     

[{'name': 'Основы концептуального мышления / Introduction to Conceptual Thinking',
  'credits': 3,
  'classroomHours': 8,
  'year': 1,
  'type': 'По выбору',
  'readBy': 'Креатив'},
 {'name': 'High Tech Business Creation: check-list for entrepreneurs / Создание технологического бизнеса: чек-лист для предпринимателей',
  'credits': 3,
  'classroomHours': 32,
  'year': 1,
  'type': 'По выбору',
  'readBy': 'Пр Кул'},
 {'name': 'High Tech Business Creation: check-list for entrepreneurs / Создание технологического бизнеса: чек-лист для предпринимателей',
  'credits': 3,
  'classroomHours': 32,
  'year': 2,
  'type': 'По выбору',
  'readBy': 'Пр Кул'},
 {'name': 'Critical Thinking Skills (Advanced)',
  'credits': 3,
  'classroomHours': 32,
  'year': 1,
  'type': 'По выбору',
  'readBy': 'F'},
 {'name': 'Critical Thinking Skills (Advanced)',
  'credits': 3,
  'classroomHours': 32,
  'year': 2,
  'type': 'По выбору',
  'readBy': 'F'},
 {'name': 'Иностранный язык / Foreign Language',
  'credit

In [ ]:
len(l)

141